## 数据准备

我们需要区分logo和其他的类别，因此我们将数据集划分为训练集、验证集和测试集，分别存放在不同的文件夹中：

```python
train_dir = './data/work1/train'
train_logo_dir = './data/work1/train_logo'
train_other_dir = './data/work1/train_other'
val_dir = './data/work1/val'
val_logo_dir = './data/work1/val_logo'
val_other_dir = './data/work1/val_other'
```


训练模型的时候你应该侧重精确率还是召回率？
在训练模型时，选择侧重精确率还是召回率取决于具体的应用场景和业务需求。
- 如果假阳性（false positive）代价较高，例如在垃圾邮件过滤中，误将正常邮件标记为垃圾邮件会导致用户体验下降，那么应该侧重提高精确率。
- 如果假阴性（false negative）代价较高，例如在疾病筛查中，漏掉患病个体可能会导致严重后果，那么应该侧重提高召回率。
- 在某些情况下，可能需要在精确率和召回率之间找到一个平衡点，这时可以使用F1分数作为评估指标。

对于logo识别任务，通常更关注召回率，因为漏掉一个logo可能会导致用户无法识别品牌，从而影响用户体验和品牌形象。因此，在训练模型时，可能会更侧重于提高召回率，以确保尽可能多的logo被正确识别出来。当然，具体的侧重点还需要根据实际应用场景进行调整。

git clone https://github.com/syuu1987/geekTime-image-classification

![image4.png](mdfiles/image4.png)

作者比较了单独放大这三个维度中的任意一个维度效果如何。得出结论是放大网络深度或网络宽度或图像分辨率，均可提升模型精度，但是越放大，精度增加越缓慢，且计算复杂度 FLOPS 增加较快。

因此，作者提出了混合维度放大法，该方法使用一个 ϕ（混合稀疏）来决定三个维度的放大倍率。

深度 
$depth：d=αϕ$

宽度 
$width：w=βϕ$

分辨率 
$resolution：r=γϕ$

$$
    s.t. α⋅β^2⋅γ^2≈2, α≥1, β≥1, γ≥1
$$

s.t. 是什么意思？
s.t. 是 "subject to" 的缩写，意思是 "在...条件下" 或 "受...约束"。在数学优化问题中，s.t. 用于引入约束条件，表示在满足这些条件的前提下进行优化。

固定 ϕ 为 1，也就是计算量为 2 倍时，使用网格搜索法找到 α、β、γ 的最优解，分别为 1.2、1.1、1.15。
然后固定 α 为 1.2，β 为 1.1，γ 为 1.15，调整 ϕ 的值来放大网络。 得到B1 到 B7 共 8 个 EfficientNet 模型。

上述操作的原因是？

通过固定其他两个维度的放大倍率，调整混合稀疏 ϕ 的值，可以在保持计算量相对稳定的情况下，探索不同的网络结构组合，从而找到最优的模型配置。

![image5.png](mdfiles/image5.png)

EfficientNet 利用一种复合的缩放手段，对网络的深度 depth、宽度 width 和分辨率 resolution 同时进行缩放（按照一定的缩放规律），来达到精度和运算复杂度 FLOPS 的权衡。

但即使只探索这三个维度，搜索空间仍然很大，所以作者规定只在 B0（作者提出的 EfficientNet 的一个 Baseline）上进行放大。

如何理解放大 深度，宽度，分辨率？
- 深度 scaling depth：增加网络的层数，使得模型能够学习到更复杂的特征表示。
- 宽度 scaling width：增加每一层的通道数，使得模型能够捕捉到更多的特征。
- 分辨率 scaling resolution：增加输入图像的分辨率，使得模型能够获取更细粒度的图像信息。
通过这种复合的缩放方法，EfficientNet 能够在保持较低计算复杂度的同时，显著提升模型的性能。

In [1]:
from torchvision import transforms
from torchvision import datasets
from torchvision.transforms import InterpolationMode

# 这个可以快速方便数据读出处理

def _norm_advprop(img):
    # 为什么 * 2.0 - 1.0 ?
    # 因为我们希望将输入图像的像素值范围从 [0, 1] 线性映射到 [-1, 1]
    return img * 2.0 - 1.0

def build_transform(dest_image_size):
    # lambda 方式实现归一化
    normalize = transforms.Lambda(_norm_advprop)

    if not isinstance(dest_image_size, tuple):
        dest_image_size = (dest_image_size, dest_image_size)
    else:
        dest_image_size = dest_image_size

    transform = transforms.Compose([
        # 先缩放再裁剪保证尺寸一致
        transforms.RandomResizedCrop(dest_image_size),
        # 随机水平翻转
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ])

    return transform

def build_data_set(dest_image_size, data):
    transform = build_transform(dest_image_size) 
    dataset=datasets.ImageFolder(data, transform=transform, target_transform=None) 

    return dataset


![image6.png](mdfiles/image6.png)

In [2]:
from efficientnet import EfficientNet
from torch.utils.data import DataLoader
from torchvision import datasets
import torch.nn as nn
import torch
import argparse
import os


def train(train_loader, model, criterion, optimizer, epoch, args, device):
    model.train()
    for i, (images, target) in enumerate(train_loader):
        # === 关键：把数据搬到同一设备 ===
        images = images.to(device, non_blocking=True)
        # CE 需要 int64 的类别索引
        target = target.to(device, non_blocking=True).long()

        # forward
        output = model(images)
        loss = criterion(output, target)
        print(f'Epoch {epoch} | step {i} | loss {loss.item():.4f}')

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

def main(args):

    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    if args.pretrained:
        # 使用预训练模型，在这里就进行微调了，num_classes 指定新的类别数量为 args.classes_num
        model = EfficientNet.from_pretrained('efficientnet-b0',
                                             num_classes=args.classes_num,
                                             advprop=args.advprop)
        print(f"=> using pre-trained model '{args.arch}'")
    else:
        print(f"=> creating model '{args.arch}'")
        model = EfficientNet.from_name(args.arch, num_classes=args.classes_num)

    model = model.to(device)

    # CrossEntropyLoss 包含了 Softmax 和 NLLLoss
    criterion = nn.CrossEntropyLoss().to(device)

    optimizer = torch.optim.SGD(model.parameters(), args.lr,
                                momentum=args.momentum, weight_decay=args.weight_decay)

    use_pin = torch.cuda.is_available()
    
    # args.train_data 指定训练数据集路径
    train_dataset = build_data_set(args.image_size, args.train_data)
    # use_pin 用于加速数据传输到 GPU
    train_loader = DataLoader(train_dataset,
                              batch_size=args.batch_size,
                              shuffle=True,
                              num_workers=args.num_workers,
                              pin_memory=use_pin)

    
    # 创建 checkpoint 目录
    os.makedirs(args.checkpoint_dir, exist_ok=True)

    # 开始训练
    for epoch in range(args.epochs):
        train(train_loader, model, criterion, optimizer, epoch, args, device)
        if epoch % args.save_interval == 0:
            torch.save(model.state_dict(),
                       os.path.join(args.checkpoint_dir, f'checkpoint.pth.tar.epoch_{epoch}'))

# Quick dataset check: detect class folders under ./data/work1/train and set detected_classes_num
# This cell is safe to run in the notebook and will help set the default number of classes.
import os
from pathlib import Path
train_path = Path('./data/work1/train')
if train_path.exists():
    # class_dirs is logo and other folders under train/, this way is to create the class_dirs automatically
    class_dirs = sorted([p.name for p in train_path.iterdir() if p.is_dir()])
    # 获取类别数量
    detected_classes_num = len(class_dirs) if class_dirs else 0
    print(f'Found train dir: {train_path}. Classes: {class_dirs} (count={detected_classes_num})')
else:
    detected_classes_num = 0
    print(f'WARNING: Train dir {train_path} not found. Please prepare training data at ./data/work1/train')
    raise FileNotFoundError(f'Train dir {train_path} not found.')

# Use a Namespace with default values so the notebook can run without CLI args.
# Edit these defaults directly in the notebook as needed.
args = argparse.Namespace(
    train_data='./data/work1/train',
    image_size=224, # 输入图片的宽和高，B0 推荐 224
    batch_size=10,  # each GPU batch size
    num_workers=4,  # data loading workers
    epochs=10,      # total training epochs
    lr=0.001,       # learning rate
    checkpoint_dir='./ckpts/', # checkpoint save path
    save_interval=1,           # save checkpoint every N epochs
    momentum=0.9,              # momentum 动量 目的是加快 SGD 优化器的收敛速度
    weight_decay=1e-4,         # 权重衰减（L2 正则化）
    arch='efficientnet-b0',    # 模型架构
    pretrained=True,           # 是否使用预训练权重
    advprop=False,             # 是否使用 advprop 预训练权重
    classes_num=detected_classes_num if detected_classes_num > 0 else 2  # default to 2 if not detected
)

main(args)

Found train dir: data/work1/train. Classes: ['logo', 'others'] (count=2)
Loaded pretrained weights for efficientnet-b0
=> using pre-trained model 'efficientnet-b0'
Epoch 0 | step 0 | loss 0.7223
Epoch 0 | step 1 | loss 0.7417
Epoch 1 | step 0 | loss 0.7709
Epoch 1 | step 1 | loss 0.8034
Epoch 2 | step 0 | loss 0.6751
Epoch 2 | step 1 | loss 0.7021
Epoch 3 | step 0 | loss 0.6279
Epoch 3 | step 1 | loss 0.6618
Epoch 4 | step 0 | loss 0.5962
Epoch 4 | step 1 | loss 0.6511
Epoch 5 | step 0 | loss 0.5748
Epoch 5 | step 1 | loss 0.5387
Epoch 6 | step 0 | loss 0.5416
Epoch 6 | step 1 | loss 0.5221
Epoch 7 | step 0 | loss 0.4599
Epoch 7 | step 1 | loss 0.6675
Epoch 8 | step 0 | loss 0.4153
Epoch 8 | step 1 | loss 0.4239
Epoch 9 | step 0 | loss 0.3326
Epoch 9 | step 1 | loss 0.3877


In [3]:
# 给我使用val数据集进行验证

# 使用 val 数据集进行验证（简洁版）
import torch
import torch.nn as nn
from pathlib import Path
from efficientnet import EfficientNet
from torch.utils.data import DataLoader

# 设备
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# 验证集 DataLoader
use_pin = torch.cuda.is_available()
# 定义一个数据集包装类，使 __getitem__ 返回 (img, label, path) 方便定位误分类样本
class ImageFolderWithPaths(datasets.ImageFolder):
    def __getitem__(self, index):
        img, target = super().__getitem__(index)
        path = self.samples[index][0]
        return img, target, path

val_dataset = ImageFolderWithPaths('./data/work1/val', transform=build_transform(args.image_size))

val_loader = DataLoader(val_dataset,
                          batch_size=args.batch_size,
                          shuffle=False,
                          num_workers=args.num_workers,
                          pin_memory=use_pin)

print('Val classes:', val_dataset.classes)
print('Val class_to_idx:', val_dataset.class_to_idx)

# how many pics in each class
class_counts = {}
for _, label in val_dataset.samples:
    class_name = val_dataset.classes[label]
    if class_name in class_counts:
        class_counts[class_name] += 1
    else:
        class_counts[class_name] = 1

print('每个类别的样本数量:', class_counts)

# 构建模型并加载最新权重
model = EfficientNet.from_name(args.arch, num_classes=args.classes_num)
ckpt_dir = Path(args.checkpoint_dir)
ckpts = sorted(ckpt_dir.glob('checkpoint.pth.tar.epoch_*'))
if not ckpts:
    raise FileNotFoundError(f'未找到权重文件，请先完成训练: {ckpt_dir}')
latest = ckpts[-1]
state = torch.load(latest, map_location=device)
model.load_state_dict(state)
model = model.to(device)
model.eval()

# 验证循环：计算Top-1准确率与平均Loss
criterion = nn.CrossEntropyLoss().to(device)
total = 0
correct = 0
loss_sum = 0.0
# 记录误分类样本 (path, true_name, pred_name)
misclassified = []
idx2class = {i: c for i, c in enumerate(val_dataset.classes)}

with torch.no_grad():
    for images, target, paths in val_loader:
        images = images.to(device, non_blocking=True)
        target = target.to(device, non_blocking=True).long()
        output = model(images)
        loss = criterion(output, target)
        loss_sum += loss.item() * images.size(0)
        pred = output.argmax(dim=1)
        correct += (pred == target).sum().item()
        # 收集误分类
        for j in range(images.size(0)):
            pi = int(pred[j].item())
            ti = int(target[j].item())
            if pi != ti:
                misclassified.append((paths[j], idx2class[ti], idx2class[pi]))
        total += target.size(0)

avg_loss = loss_sum / max(total, 1)
acc = correct / max(total, 1)
print(f'验证集: 样本 {total} | Loss {avg_loss:.4f} | Top-1 准确率 {acc*100:.2f}% | 来自权重: {latest.name}')

# 输出误分类样本名称（显示前 50 条，避免过长）
print(f'误分类样本数: {len(misclassified)}')
for p, tname, pname in misclassified[:50]:
    print(f'{p} | 真: {tname} -> 预测: {pname}')
if len(misclassified) > 50:
    print(f'... 仅显示前 50 条，共 {len(misclassified)} 条')


# 我引入的没有logo的分别放在了 val 的 logo 和 other 文件夹下
# 这个被误判为logo
# 这个模型真正学习到的东西是什么呢，需要仔细考虑一下

Val classes: ['logo', 'others']
Val class_to_idx: {'logo': 0, 'others': 1}
每个类别的样本数量: {'logo': 17, 'others': 11}


/tmp/ipykernel_1644670/3405174438.py:51: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(latest, map_location=device)


验证集: 样本 28 | Loss 0.4378 | Top-1 准确率 96.43% | 来自权重: checkpoint.pth.tar.epoch_9
误分类样本数: 1
./data/work1/val/others/11.jpeg | 真: others -> 预测: logo
